# Orbit (**O**bject-**OR**iented **B**ayes**I**an **T**ime Series)

- [Orbit: A Python Package for Bayesian Forecasting](https://github.com/uber/orbit/tree/master)
- [Orbit’s Documentation](https://orbit-ml.readthedocs.io/en/stable/)
- [Quick Start](https://orbit-ml.readthedocs.io/en/stable/tutorials/quick_start.html#)
- [Orbit: Probabilistic Forecast with Exponential Smoothing](https://arxiv.org/abs/2004.08492) Paper


### Implemented Models

- ETS (which stands for Error, Trend, and Seasonality) Model
- Methods of Estimations
    - Maximum a Posteriori (MAP)
    - Full Bayesian Estimation
    - Aggregated Posteriors
- Damped Local Trend (DLT)
    - Global Trend Configurations:
        - Linear Global Trend
        - Log-Linear Global Trend
        - Flat Global Trend
        - Logistic Global Trend
    - Damped Local Trend Full Bayesian Estimation (DLTFull)
- Local Global Trend (LGT)
    - Local Global Trend Maximum a Posteriori (LGTMAP)
    - Local Global Trend for full Bayesian prediction (LGTFull)
    - Local Global Trend for aggregated posterior prediction (LGTAggregated)
- Using Pyro for Estimation
    - MAP Fit and Predict
    - VI Fit and Predict
- Kernel-based Time-varying Regression (KTR)
    - Kernel-based Time-varying Regression Lite (KTRLite)

In [ ]:
!pip install orbit-ml --no-input

In [ ]:
import awswrangler as wr
import boto3
from sagemaker import get_execution_role
import pandas as pd
import numpy as np
import json
import urllib3
from scipy.optimize import curve_fit

import orbit
from orbit import *
from orbit.models.dlt import ETSFull, ETSMAP, ETSAggregated, DLTMAP, DLTFull, DLTMAP, DLTAggregated
from orbit.models.lgt import LGTMAP, LGTAggregated, LGTFull
from orbit.models.ktrlite import KTRLiteMAP

from orbit.estimators.pyro_estimator import PyroEstimatorVI, PyroEstimatorMAP

In [ ]:
import warnings

warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')

# Uploading data

- uploading data for **models**

In [ ]:
role = get_execution_role()
bucket='...'
data_key = '...csv' 
data_location = 's3://{}/{}'.format(bucket, data_key)

In [ ]:
df = pd.DataFrame(pd.read_csv(data_location))

In [ ]:
df = df.rename({'Unnamed: 0': 'Date'}, axis = 1)
df.index = df['Date']

In [ ]:
df.shape

In [ ]:
#df = df.drop(['curve'], axis = 0)

In [ ]:
df

### Orbit Models

In [ ]:
# ETS (which stands for Error, Trend, and Seasonality)

# Methods of Estimations

# Maximum a Posteriori (MAP)

# The advantage of MAP estimation is a faster computational speed.

def ETSMAP_model(date_col, response_col, train_df, test_df):
    ets = ETSMAP(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
    )
    
    ets.fit(df=train_df)
    predicted_df_MAP = ets.predict(df=test_df)
    
    return predicted_df_MAP['prediction']

# Full Bayesian Estimation


def ETSFull_model(date_col, response_col, train_df, test_df):
    ets = ETSFull(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
        num_warmup=400,
        num_sample=400,
    )
    
    ets.fit(df=train_df)
    predicted_df_ETSFull = ets.predict(df=test_df)
    
    return predicted_df_ETSFull['prediction']

# Aggregated Posteriors

def ETSAggregated_model(date_col, response_col, train_df, test_df):
    ets = ETSAggregated(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
    )
    ets.fit(df=train_df)
    predicted_df_ETSAggregated = ets.predict(df=test_df)
    
    return predicted_df_ETSAggregated['prediction']


# Damped Local Trend (DLT)

# Global Trend Configurations

# Linear Global Trend

# linear global trend
def DLTMAP_lin(date_col, response_col, train_df, test_df):
    dlt = DLTMAP(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
    )

    dlt.fit(train_df)
    predicted_df_DLTMAP_lin = dlt.predict(test_df)
    
    return predicted_df_DLTMAP_lin['prediction']


# log-linear global trend
def DLTMAP_log_lin(date_col, response_col, train_df, test_df):
    dlt = DLTMAP(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
        global_trend_option='loglinear'
    )

    dlt.fit(train_df)
    predicted_df_DLTMAP_log_lin = dlt.predict(test_df)
    
    return predicted_df_DLTMAP_log_lin['prediction']


# log-linear global trend
def DLTMAP_flat(date_col, response_col, train_df, test_df):
    dlt = DLTMAP(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
        global_trend_option='flat'
    )

    dlt.fit(train_df)
    predicted_df_DLTMAP_flat = dlt.predict(test_df)
    
    return predicted_df_DLTMAP_flat['prediction']


# logistic global trend
def DLTMAP_logistic(date_col, response_col, train_df, test_df):
    dlt = DLTMAP(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
        global_trend_option='logistic'
    )

    dlt.fit(train_df)
    predicted_df_DLTMAP_logistic = dlt.predict(test_df)
    
    return predicted_df_DLTMAP_logistic['prediction']


# Damped Local Trend Full Bayesian Estimation (DLTFull)

def DLTFull_model(date_col, response_col, train_df, test_df):
    dlt = DLTFull(
        response_col=response_col,
        date_col=date_col,
        num_warmup=400,
        num_sample=400,
        seasonality=52,
        seed=8888
    )
    
    dlt.fit(df=train_df)
    predicted_df_DLTFull = dlt.predict(df=test_df)

    return predicted_df_DLTFull['prediction']


# Damped Local Trend Full (DLTAggregated)

def DLTAggregated_model(date_col, response_col, train_df, test_df):
    ets = DLTAggregated(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
    )
    
    ets.fit(df=train_df)
    predicted_df_DLTAggregated = ets.predict(df=test_df)
    
    return predicted_df_DLTAggregated['prediction']


# Local Global Trend (LGT) Model

# Local Global Trend Maximum a Posteriori (LGTMAP)

def LGTMAP_model(date_col, response_col, train_df, test_df):
    lgt = LGTMAP(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
    )

    lgt.fit(df=train_df)
    predicted_df_LGTMAP = lgt.predict(df=test_df)
    
    return predicted_df_LGTMAP['prediction']

# LGTFull

def LGTFull_model(date_col, response_col, train_df, test_df):
    lgt = LGTFull(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
    )

    lgt.fit(df=train_df)
    predicted_df_LGTFull = lgt.predict(df=test_df)
    
    return predicted_df_LGTFull['prediction']

# LGTAggregated

def LGTAggregated_model(date_col, response_col, train_df, test_df):
    lgt = LGTAggregated(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
    )

    lgt.fit(df=train_df)
    predicted_df_LGTAggregated = lgt.predict(df=test_df)
    
    return predicted_df_LGTAggregated['prediction']

# Using Pyro for Estimation

# MAP Fit and Predict

def LGTMAP_PyroEstimatorMAP(date_col, response_col, train_df, test_df):
    lgt_map = LGTMAP(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
        estimator_type=PyroEstimatorMAP,
    )

    lgt_map.fit(df=train_df)
    predicted_df_LGTMAP_pyro = lgt_map.predict(df=test_df)
    
    return predicted_df_LGTMAP_pyro['prediction']

# VI Fit and Predict

def LGTFull_pyro(date_col, response_col, train_df, test_df):
    lgt_vi = LGTFull(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
        num_steps=101,
        num_sample=100,
        learning_rate=0.1,
        n_bootstrap_draws=-1,
        estimator_type=PyroEstimatorVI,
    )

    lgt_vi.fit(df=train_df)

    predicted_df_LGTFull_pyro = lgt_vi.predict(df=test_df)
    
    return predicted_df_LGTFull_pyro['prediction']


# Kernel-based Time-varying Regression (KTR)

# KTRLite

def ktrlite_MAP(date_col, response_col, train_df, test_df):
    ktrlite = KTRLiteMAP(
        response_col=response_col,
        #response_col=np.log(df[response_col]),
        date_col=date_col,
        level_knot_scale=.1,
        span_level=.05,
    )
    
    ktrlite.fit(train_df)
    
    predicted_df_ktrlite_MAP = ktrlite.predict(df=test_df, decompose=True)
    
    return predicted_df_ktrlite_MAP['prediction']

### Root-Mean-Square Deviation (RMSD) or Root-Mean-Square Error (RMSE)

In [ ]:
def rmse(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.sqrt(np.square(np.subtract(actual,pred)).mean())

In [ ]:
def evaluating_models(index, column):
    
    '''
    Parameters:
        index: column index
        column: column name
    
    Returns:
        models_df: new dataframe with 
    '''

    cocompared_values = compared_function(column) # the values with which the model values are compared

    tmp_df['Date'] = pd.to_datetime(df['Date'].astype(str))
    tmp_df['Penetration'] = df[column].astype(float)
    
    date_col = 'Date'
    response_col = 'Penetration'
    
    # Forecasting for N years ahead
    
    test_size = 4 # 5 - without 2021, 4 - with 2021
    train_df = tmp_df[:-test_size]
    test_df = tmp_df[-test_size:]

    # Decompose Prediction

    #train_df = tmp_df[tmp_df['Date'] < '2022-01-01']
    #test_df = tmp_df[tmp_df['Date'] <= '2025-01-01']
    
    models_df.at[index ,'Item Name'] = column

    
    # Making predictions with each model
    try:
        models_df.at[index , 'ETSMAP'] = rmse(
            cocompared_values, (ETSMAP_model(date_col, response_col, train_df, test_df))).astype(float)
    except:
        models_df.at[index , 'ETSMAP'] = 100
    try:    
        models_df.at[index , 'ETSFull'] = rmse(
            cocompared_values, (ETSFull_model(date_col, response_col, train_df, test_df))).astype(float)
    except:
        models_df.at[index , 'ETSFull'] = 100
    try:
        models_df.at[index , 'ETSAggregated'] = rmse(
            cocompared_values, (ETSAggregated_model(date_col, response_col, train_df, test_df))).astype(float)
    except:
        models_df.at[index , 'ETSAggregated'] = 100

    
    try:
        models_df.at[index , 'DLTMAP_lin'] = rmse(
            cocompared_values, (DLTMAP_lin(date_col, response_col, train_df, test_df))).astype(float)
    except:
        models_df.at[index , 'DLTMAP_lin'] = 100
    try:
        models_df.at[index , 'DLTMAP_log_lin'] = rmse(
            cocompared_values, (DLTMAP_log_lin(date_col, response_col, train_df, test_df))).astype(float)
    except:
        models_df.at[index , 'DLTMAP_log_lin'] = 100
    try:
        models_df.at[index , 'DLTMAP_flat'] = rmse(
            cocompared_values, (DLTMAP_flat(date_col, response_col, train_df, test_df))).astype(float)
    except:
        models_df.at[index , 'DLTMAP_flat'] = 100
    try:
        models_df.at[index , 'DLTMAP_logistic'] = rmse(
            cocompared_values, (DLTMAP_logistic(date_col, response_col, train_df, test_df))).astype(float)
    except:
        models_df.at[index , 'DLTMAP_logistic'] = 100
    try:    
        models_df.at[index , 'DLTFull'] = rmse(
            cocompared_values, (DLTFull_model(date_col, response_col, train_df, test_df))).astype(float)
    except:
        models_df.at[index , 'DLTFull'] = 100
    try:
        models_df.at[index , 'DLTAggregated'] = rmse(
            cocompared_values, (DLTAggregated_model(date_col, response_col, train_df, test_df))).astype(float)
    except:  
        models_df.at[index , 'DLTAggregated'] = 100
    
    
    try:
        models_df.at[index , 'LGTMAP'] = rmse(
            cocompared_values, (LGTMAP_model(date_col, response_col, train_df, test_df))).astype(float)
    except:
        models_df.at[index , 'LGTMAP'] = 100
    try:
        models_df.at[index , 'LGTFull'] = rmse(
            cocompared_values, (LGTFull_model(date_col, response_col, train_df, test_df))).astype(float)
    except: 
        models_df.at[index , 'LGTFull'] = 100
    try: 
        models_df.at[index , 'LGTAggregated'] = rmse(
            cocompared_values, (LGTAggregated_model(date_col, response_col, train_df, test_df))).astype(float)
    except:
        models_df.at[index , 'LGTAggregated'] = 100

    
    # Using Pyro for Estimation
    try:
        models_df.at[index , 'LGTMAP_PyroEstimatorMAP'] = rmse(
            cocompared_values, (LGTMAP_PyroEstimatorMAP(date_col, 
                                                    response_col, train_df, test_df))).astype(float)
    except:
        models_df.at[index , 'LGTMAP_PyroEstimatorMAP'] = 100
    try:
        models_df.at[index , 'LGTFull_pyro4'] = rmse(
            cocompared_values, (LGTFull_pyro(date_col, response_col, train_df, test_df))).astype(float)
    except:
         models_df.at[index , 'LGTFull_pyro4'] = 100
        
    # Kernel-based Time-varying Regression (KTR)
    try:
        models_df.at[index , 'KTR_Lite_MAP'] = rmse(
            cocompared_values, (ktrlite_MAP(date_col, response_col, train_df, test_df))).astype(float)
    except:
        models_df.at[index , 'KTR_Lite_MAP'] = 100
    
    
    models_df.at[index, 'Curve Type'] = df[column].iloc[-1]
        
        
    return models_df

### Calculating minimal RMSE value for each item

In [ ]:
def min_value(df):
    
    '''
    Parameters:
        df: input dataframe with multiple columns and values in a row
    
    Returns:
        models_df: existing dataframe with added the new 'Model' column filled with 
        the name of the best-fitted model for each item
    '''
        
    df.iloc[:, 1:-1].apply(pd.to_numeric)
    df['Model'] = df.iloc[:, 1:-1].idxmin(axis=1)
    
    return models_df

### Evaluating Orbit models for each item

In [ ]:
import time


tmp_df = pd.DataFrame()
models_df = pd.DataFrame()

start = time.time()

for index, column in enumerate(curve_df.columns[1:2]):
    evaluating_models(index, column)
    
end = time.time()
print(end - start)

In [ ]:
models_df

In [ ]:
min_value(models_df)

# 

-------------------------

## ETS (which stands for Error, Trend, and Seasonality)

### Methods of Estimations

#### Maximum a Posteriori (MAP)

> The advantage of MAP estimation is a **faster computational speed**.

In [ ]:
def ETSMAP_model(date_col, response_col, train_df, test_df):
    ets = ETSMAP(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
    )
    
    ets.fit(df=train_df)
    predicted_df_MAP = ets.predict(df=test_df)
    
    return predicted_df_MAP['prediction']

## Full Bayesian Estimation

>  Compared to MAP, it usually takes longer time to fit a full Bayesian models where **No-U-Turn Sampler (NUTS)** ([Hoffman and Gelman 2011](https://arxiv.org/abs/1111.4246)) is carried out under the hood. The advantage is that the inference and estimation are usually more robust.

In [ ]:
def ETSFull_model(date_col, response_col, train_df, test_df):
    ets = ETSFull(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
        num_warmup=400,
        num_sample=400,
    )
    
    ets.fit(df=train_df)
    predicted_df_ETSFull = ets.predict(df=test_df)
    
    return predicted_df_ETSFull['prediction']

## Aggregated Posteriors

> Just like the full Bayesian method, it runs through the MCMC algorithm which is **NUTS** by default. The difference from a full model is that aggregated model first aggregates the posterior samples based on mean or median (via aggregate_method) then does the prediction using the aggreated posterior.

In [ ]:
def ETSAggregated_model(date_col, response_col, train_df, test_df):
    ets = ETSAggregated(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
    )
    ets.fit(df=train_df)
    predicted_df_ETSAggregated = ets.predict(df=test_df)
    
    return predicted_df_ETSAggregated['prediction']

## Damped Local Trend (DLT)

> DLT is one of the main exponential smoothing models support in `orbit`. Performance is benchmarked with M3 monthly, M4 weekly dataset and some Uber internal dataset ([Ng and Wang et al., 2020](https://arxiv.org/abs/2004.08492)). The model is a fusion between the classical ETS ([Hyndman et. al., 2008](http://www.exponentialsmoothing.net/home))) with some refinement leveraging ideas from Rlgt ([Smyl et al., 2019](https://cran.r-project.org/web/packages/Rlgt/index.html)). 

### Global Trend Configurations

#### Linear Global Trend

In [ ]:
# linear global trend
def DLTMAP_lin(date_col, response_col, train_df, test_df):
    dlt = DLTMAP(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
    )

    dlt.fit(train_df)
    predicted_df_DLTMAP_lin = dlt.predict(test_df)
    
    return predicted_df_DLTMAP_lin['prediction']

In [ ]:
# log-linear global trend
def DLTMAP_log_lin(date_col, response_col, train_df, test_df):
    dlt = DLTMAP(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
        global_trend_option='loglinear'
    )

    dlt.fit(train_df)
    predicted_df_DLTMAP_log_lin = dlt.predict(test_df)
    
    return predicted_df_DLTMAP_log_lin['prediction']

In [ ]:
# log-linear global trend
def DLTMAP_flat(date_col, response_col, train_df, test_df):
    dlt = DLTMAP(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
        global_trend_option='flat'
    )

    dlt.fit(train_df)
    predicted_df_DLTMAP_flat = dlt.predict(test_df)
    
    return predicted_df_DLTMAP_flat['prediction']

#### Logistic Global Trend

In [ ]:
# logistic global trend
def DLTMAP_logistic(date_col, response_col, train_df, test_df):
    dlt = DLTMAP(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
        global_trend_option='logistic'
    )

    dlt.fit(train_df)
    predicted_df_DLTMAP_logistic = dlt.predict(test_df)
    
    return predicted_df_DLTMAP_logistic['prediction']

### Damped Local Trend Full Bayesian Estimation (DLTFull)

In [ ]:
def DLTFull_model(date_col, response_col, train_df, test_df):
    dlt = DLTFull(
        response_col=response_col,
        date_col=date_col,
        num_warmup=400,
        num_sample=400,
        seasonality=52,
        seed=8888
    )
    
    dlt.fit(df=train_df)
    predicted_df_DLTFull = dlt.predict(df=test_df)

    return predicted_df_DLTFull['prediction']

### Damped Local Trend Full (DLTAggregated)

In [ ]:
def DLTAggregated_model(date_col, response_col, train_df, test_df):
    ets = DLTAggregated(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
    )
    
    ets.fit(df=train_df)
    predicted_df_DLTAggregated = ets.predict(df=test_df)
    
    return predicted_df_DLTAggregated['prediction']

## Local Global Trend (LGT) Model

> LGT stands for Local and Global Trend and is a refined model from **Rlgt** ([Smyl et al., 2019](https://cran.r-project.org/web/packages/Rlgt/index.html)). The main difference is that LGT is an additive form taking log-transformation response as the modeling response. This essentially converts the model into a multicplicative with some advantages ([Ng and Wang et al., 2020](https://arxiv.org/abs/2004.08492)). 

### Local Global Trend Maximum a Posteriori (LGTMAP)

> LGTMAP is the model class for MAP (Maximum a Posteriori) estimation.

In [ ]:
def LGTMAP_model(date_col, response_col, train_df, test_df):
    lgt = LGTMAP(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
    )

    lgt.fit(df=train_df)
    predicted_df_LGTMAP = lgt.predict(df=test_df)
    
    return predicted_df_LGTMAP['prediction']

### LGTFull

> LGTFull is the model class for full Bayesian prediction. In full Bayesian prediction, the prediction will be conducted once for each parameter posterior sample, and the final prediction results are aggregated. Prediction will always return the median (aka 50th percentile) along with any additional percentiles that are provided.

In [ ]:
def LGTFull_model(date_col, response_col, train_df, test_df):
    lgt = LGTFull(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
    )

    lgt.fit(df=train_df)
    predicted_df_LGTFull = lgt.predict(df=test_df)
    
    return predicted_df_LGTFull['prediction']

#### Observations

- **Time:** This model takes longer time to fit a full Bayesian models where No-U-Turn Sampler (NUTS) ([Hoffman and Gelman 2011](https://arxiv.org/abs/1111.4246)) is carried out under the hood.

### LGTAggregated

> LGTAggregated is the model class for aggregated posterior prediction. In aggregated prediction, the parameter posterior samples are reduced using `aggregate_method ({ 'mean', 'median' })` before performing a single prediction.

In [ ]:
def LGTAggregated_model(date_col, response_col, train_df, test_df):
    lgt = LGTAggregated(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
    )

    lgt.fit(df=train_df)
    predicted_df_LGTAggregated = lgt.predict(df=test_df)
    
    return predicted_df_LGTAggregated['prediction']

## Using Pyro for Estimation

> Currently are still experimenting Pyro and support Pyro only with LGT.

### MAP Fit and Predict

In [ ]:
def LGTMAP_PyroEstimatorMAP(date_col, response_col, train_df, test_df):
    lgt_map = LGTMAP(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
        estimator_type=PyroEstimatorMAP,
    )

    lgt_map.fit(df=train_df)
    predicted_df_LGTMAP_pyro = lgt_map.predict(df=test_df)
    
    return predicted_df_LGTMAP_pyro['prediction']

### VI Fit and Predict

In [ ]:
def LGTFull_pyro(date_col, response_col, train_df, test_df):
    lgt_vi = LGTFull(
        response_col=response_col,
        date_col=date_col,
        seasonality=52,
        seed=8888,
        num_steps=101,
        num_sample=100,
        learning_rate=0.1,
        n_bootstrap_draws=-1,
        estimator_type=PyroEstimatorVI,
    )

    lgt_vi.fit(df=train_df)

    predicted_df_LGTFull_pyro = lgt_vi.predict(df=test_df)
    
    return predicted_df_LGTFull_pyro['prediction']

## Kernel-based Time-varying Regression (KTR)

> Implemented in the stable version of Orbit library
>
> [Documentation](https://orbit-ml.readthedocs.io/en/stable/tutorials/ktrlite.html)

### KTRLite

In [ ]:
def ktrlite_MAP(date_col, response_col, train_df, test_df):
    ktrlite = KTRLiteMAP(
        response_col=response_col,
        #response_col=np.log(df[response_col]),
        date_col=date_col,
        level_knot_scale=.1,
        span_level=.05,
    )
    
    ktrlite.fit(train_df)
    
    predicted_df_ktrlite_MAP = ktrlite.predict(df=test_df, decompose=True)
    
    return predicted_df_ktrlite_MAP['prediction']